In [1]:
import psutil
import os
p = psutil.Process(os.getpid())
p.cpu_affinity([0])

import time
start_time = time.time()

In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from QL_env_agent_e1 import ChemicalSpace, QLAgent

In [3]:
env = ChemicalSpace()
agent = QLAgent()

episodes = 10000
np.random.seed(0)

smiles_list = []
rewards = []
for episode in range(episodes):
    state = env.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.update(state, action, reward, next_state, done)
        if env.op_count > 1:
            mol = Chem.MolFromSmiles(env.smiles)
            if mol is not None:
                smiles_list.append(env.smiles)
                rewards.append(reward)
            
        if done:
            break

        state = next_state

df0 = pd.DataFrame({'SMILES': smiles_list, 'reward': rewards})

In [4]:
np.random.seed(1)

smiles_list = []
rewards = []
for episode in range(episodes):
    state = env.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.update(state, action, reward, next_state, done)
        if env.op_count > 1:
            mol = Chem.MolFromSmiles(env.smiles)
            if mol is not None:
                smiles_list.append(env.smiles)
                rewards.append(reward)
            
        if done:
            break

        state = next_state

df1 = pd.DataFrame({'SMILES': smiles_list, 'reward': rewards})

In [5]:
np.random.seed(2)

smiles_list = []
rewards = []
for episode in range(episodes):

    state = env.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.update(state, action, reward, next_state, done)
        if env.op_count > 1:
            mol = Chem.MolFromSmiles(env.smiles)
            if mol is not None:
                smiles_list.append(env.smiles)
                rewards.append(reward)
            
        if done:
            break

        state = next_state

df2 = pd.DataFrame({'SMILES': smiles_list, 'reward': rewards})

In [6]:
np.random.seed(3)

smiles_list = []
rewards = []
for episode in range(episodes):
    state = env.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.update(state, action, reward, next_state, done)
        if env.op_count > 1:
            mol = Chem.MolFromSmiles(env.smiles)
            if mol is not None:
                smiles_list.append(env.smiles)
                rewards.append(reward)
            
        if done:
            break

        state = next_state

df3 = pd.DataFrame({'SMILES': smiles_list, 'reward': rewards})

In [7]:
def number_bond_silicon(mol):
    if mol:
        for atom in mol.GetAtoms():
            if atom.GetSymbol() == 'Si':
                if atom.GetDegree() > 3:
                    return False
                if atom.GetDegree() == 2 and atom.GetNumRadicalElectrons() > 0:
                    return False
    return True

incorrect_patterns = ["[SiH2]([*])([*])([*])"]

def remove_incorrect_silicon(molecules, patterns):
    filtered_molecules = []
    for mol in molecules:
        if mol is None:
            continue
        remove = False
        for pattern in patterns:
            patt = Chem.MolFromSmarts(pattern)
            if mol.HasSubstructMatch(patt):
                remove = True
                break
        if not remove:
            filtered_molecules.append(mol)
    return filtered_molecules

def number_bond_phosphorus(mol):
    if mol:
        for atom in mol.GetAtoms():
            if atom.GetSymbol() == 'P':
                if atom.GetDegree() > 4 and atom.GetNumExplicitHs() > 0:
                    return False
    return True


data = pd.concat([df0, df1, df2, df3]).reset_index(drop=True)
data = data[data['reward'] > 0]

smiles_list = np.array(data['SMILES'])
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

mols = [mol for mol in mols if number_bond_silicon(mol)]
mols = [mol for mol in mols if number_bond_phosphorus(mol)]
mols = remove_incorrect_silicon(mols, incorrect_patterns)

unique_mols = {}
for mol in mols:
    canonical_smiles = Chem.MolToSmiles(mol)
    unique_mols[canonical_smiles] = mol

final_smiles_list = list(unique_mols.keys())

f_smiles = pd.DataFrame(final_smiles_list, columns=['SMILES'])

In [8]:
def get_bertzct(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is not None:
        return Descriptors.BertzCT(molecule)
    else:
        return None

f_smiles['BertzCT'] = f_smiles['SMILES'].apply(get_bertzct)
f_data = f_smiles.dropna(subset=['BertzCT'])
f_data.to_csv('result/result_ta.csv', index=False)

In [9]:
end_time = time.time()

In [10]:
elapsed_time = end_time - start_time
print(f"run time: {elapsed_time:.0f}sec")

run time: 3007sec
